In [ ]:
import boto3
import nltk
import openai
import pandas as pd
from io import StringIO

# Constants and configurations
bucket_name = 'your-bucket-name'
s3_filename = 'book_to_summarize'
openai.api_key = 'your-api-key'
s3 = boto3.client('s3')
comprehend = boto3.client('comprehend', region_name='us-west-2')


In [ ]:
def detect_sentiment_scores(text):
    response = comprehend.detect_sentiment(Text=text, LanguageCode='en')
    scores = response['SentimentScore']
    return scores

def detect_entities(text):
    response = comprehend.detect_entities(Text=text, LanguageCode='en')
    entities = response['Entities']
    names_and_places = [entity['Text'] for entity in entities if entity['Type'] in ['PERSON', 'LOCATION']]
    return ', '.join(names_and_places)

def detect_sentiment(text):
    response = comprehend.detect_sentiment(Text=text, LanguageCode='en')
    return response['Sentiment']

def generate_summary(text, prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{prompt}:\n\n{text}"}
        ],
        max_tokens=2500,
        n=1,
        temperature=0.7,
    )
    summary = response.choices[0].message.content.strip()
    return summary

def generate_meta_summary(summaries):
    text = "\n\n".join(summaries)
    prompt = "Create a meta summary of these summaries"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{prompt}:\n\n{text}"}
        ],
        max_tokens=2500,
        n=1,
        temperature=0.7,
    )
    meta_summary = response.choices[0].message.content.strip()
    return meta_summary

def process_text_chunk(bucket_name, chunk, counter):
    filename = f'book_url_chunk{counter}.txt'

    # Write the chunk to a temporary file
    with open(filename, 'w') as f:
        f.write(' '.join(chunk))

    # Upload the file to S3
    s3.upload_file(Filename=filename, Bucket=bucket_name, Key=filename)

    # Get the file content
    file_obj = s3.get_object(Bucket=bucket_name, Key=filename)
    file_content = file_obj['Body'].read().decode('utf-8')

    # Generate chunksummary
    prompt = "Create a summary of this text"
    chunksummary = generate_summary(file_content, prompt)

    return (filename, chunksummary)


In [ ]:
file_obj = s3.get_object(Bucket=bucket_name, Key=s3_filename)
file_content = file_obj['Body'].read().decode('utf-8')

tokens = nltk.word_tokenize(file_content)
